In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
from enum import auto
from pathlib import Path
from typing import Callable, Dict, Optional, Tuple, Type, Union
import numpy as np
import pandas as pd
import rich
import torch
import typer
from torchmetrics import (
    ErrorRelativeGlobalDimensionlessSynthesis,
    MeanSquaredError,
    MetricCollection,
    PeakSignalNoiseRatio,
    StructuralSimilarityIndexMeasure,
)

from nn_core.common import PROJECT_ROOT
from rae.modules.enumerations import Output
from rae.pl_modules.pl_gautoencoder import LightningAutoencoder
from rae.utils.evaluation import parse_checkpoint_id, parse_checkpoints_tree, parse_checkpoint
from collections import defaultdict

try:
    # be ready for 3.10 when it drops
    from enum import StrEnum
except ImportError:
    from backports.strenum import StrEnum

from rae.utils.evaluation import plot_latent_space
import matplotlib.pyplot as plt
from tueplots import bundles
from tueplots import figsizes

logging.getLogger().setLevel(logging.ERROR)


BATCH_SIZE = 256


EXPERIMENT_ROOT = PROJECT_ROOT / "experiments" / "fig:latent-rotation-comparison"
EXPERIMENT_CHECKPOINTS = EXPERIMENT_ROOT / "checkpoints"

checkpoints, RUNS = parse_checkpoints_tree(EXPERIMENT_CHECKPOINTS)

PL_MODULE = LightningAutoencoder
MODEL = checkpoints["mnist"]["ae"][0]
MODEL

## Data

In [ ]:
from rae.utils.evaluation import get_dataset

images = []
targets = []
indexes = []
classes = []

from pytorch_lightning import seed_everything

seed_everything(0)

val_dataset = get_dataset(pl_module=PL_MODULE, ckpt=MODEL)
K = 512
idxs = torch.randperm(len(val_dataset))[:K]

for idx in idxs:
    sample = val_dataset[idx]
    indexes.append(sample["index"].item())
    images.append(sample["image"])
    targets.append(sample["target"])
    classes.append(sample["class"])

images_batch = torch.stack(images, dim=0)
images_batch.shape

In [ ]:
model, _ = parse_checkpoint(
    module_class=PL_MODULE,
    checkpoint_path=MODEL,
    map_location="cpu",
)
anchors_batch = model.metadata.anchors_images
anchors_batch.shape

## Plotting

In [ ]:
def plot_row(
    axes,
    dfs,
    title=None,
    equal=True,
):
    for j, ax in enumerate(axes):
        if j == 0:
            if title is not None:
                ax.set_title(title)
        if equal:
            ax.set_aspect("equal")
        plot_latent_space(ax, dfs[j], targets=[0, 1], size=0.75, bg_alpha=0.25, alpha=1, cmap=cmap, norm=norm)
    return fig

In [ ]:
def latents_distance(latents, return_mean: bool = True):
    dists = []
    all_dists = []

    for i in range(len(latents)):
        for j in range(i + 1, len(latents)):
            x = latents[i][1]
            y = latents[j][1]
            # dist = ((x - y)**2).sum(dim=-1).sqrt().mean()
            dist = F.pairwise_distance(x, y, p=2).mean()
            # dist = F.mse_loss(x, y, reduction="mean")
            # dist = ((x - y) ** 2).mean(dim=-1).mean()
            dists.append(f"pair=({i}, {j}): {dist:.2e}")
            all_dists.append(dist)
    if return_mean:
        return f"{sum(all_dists) / len(all_dists):.2e}"
    else:
        return " ".join(dists)

## Latents

In [ ]:
from rae.utils.evaluation import parse_checkpoint
import torch.nn.functional as F
from sklearn.decomposition import PCA


def get_latents(images_batch, model, key=Output.DEFAULT_LATENT):
    latents = model(images_batch)[key].detach().squeeze()
    return latents


def to_df(latents, fit_pca: bool = False):
    if fit_pca:
        latents2d = PCA(n_components=2).fit_transform(latents.cpu())
    else:
        latents2d = latents[:, [0, 1]]
    df = pd.DataFrame(
        {
            "x": latents2d[:, 0].tolist(),
            "y": latents2d[:, 1].tolist(),
            "class": classes,
            "target": targets,
            "index": indexes,
        }
    )
    return df

In [ ]:
images_latents = get_latents(images_batch, model)
anchors_latents = get_latents(anchors_batch, model)

In [ ]:
images_latents.shape, anchors_latents.shape

## Build image

In [ ]:
from rae.modules.attention import *
from tqdm import tqdm
import torch
from torch.optim.adam import Adam
from sklearn.decomposition import PCA
from scipy.stats import ortho_group

seed_everything(0)


def transform_latents(x, seed=0):
    opt_isometry, _ = TRANSFORMS[seed]

    noise = torch.randn_like(x) * 0.0001
    o = x + noise
    o = o @ opt_isometry
    return o


LIM = 6
N_ROWS = 3
N_COLS = LIM

TRANSFORMS = [
    (
        torch.tensor(ortho_group.rvs(images_latents.shape[-1]), dtype=torch.float),
        torch.zeros(images_latents.shape[-1], dtype=torch.float),
    )
    for _ in range(LIM)
]

plt.rcParams.update(bundles.icml2022())
plt.rcParams.update(figsizes.icml2022_full(ncols=N_COLS, nrows=N_ROWS, height_to_width_ratio=1.0))

cmap = plt.cm.get_cmap("Set1", 10)
norm = plt.Normalize(min(targets), max(targets))


SHARE_X = True
SHARE_Y = True
DPI = 300

fig, [
    abs_axes,
    rel_axes,
    quant_axes1,
] = plt.subplots(dpi=DPI, nrows=N_ROWS, ncols=N_COLS, sharey=SHARE_Y, sharex=SHARE_X, squeeze=True)


all_images_latents = [
    images_latents,
    *[transform_latents(images_latents, seed=idx) for idx in range(LIM - 1)],
]
all_anchors_latents = [
    anchors_latents,
    *[transform_latents(anchors_latents, seed=idx) for idx in range(LIM - 1)],
]


plot_row(
    abs_axes,
    [to_df(x) for x in all_images_latents],
)


dists_str = [f"AE; mean L2 dists: {latents_distance(all_images_latents)}"]

for axes, quant_mode, bin_size in (
    (rel_axes, None, None),
    (quant_axes1, SimilaritiesQuantizationMode.DIFFERENTIABLE_ROUND, 0.1),
    #   (quant_axes2, SimilaritiesQuantizationMode.DIFFERENTIABLE_ROUND, 0.3),
):
    rel_latents = []
    rel_attention = RelativeAttention(
        n_anchors=anchors_batch.shape,
        n_classes=len(set(targets)),
        similarity_mode=RelativeEmbeddingMethod.INNER,
        values_mode=ValuesMethod.SIMILARITIES,
        normalization_mode=NormalizationMode.L2,
        output_normalization_mode=OutputNormalization.NONE,
        similarities_quantization_mode=quant_mode,
        similarities_bin_size=bin_size,
    )
    assert sum(x.numel() for x in rel_attention.parameters()) == 0

    rel_latents = [
        rel_attention(x=img_latents, anchors=a_latents)[AttentionOutput.SIMILARITIES]
        for (img_latents, a_latents) in zip(all_images_latents, all_anchors_latents)
    ]
    plot_row(
        axes,
        [to_df(x) for x in rel_latents],
    )
    dists_str.append(f"Att, bin size: {bin_size}; mean L2 dists: {latents_distance(rel_latents)}")
dists_str

In [ ]:
fig.savefig("epsisometry.svg", bbox_inches="tight")
!rsvg-convert -f pdf -o 'epsisometry.pdf' 'epsisometry.svg'
!rm 'epsisometry'.svg